#  Проект. Рекомендация тарифов

## Описание проекта

Имеются данные о поведении клиентов, которые уже перешли на тарифы "Ультра" и "Смарт" (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

### Цели проекта
* Построить модель с максимально большим значением *accuracy*
* Довести долю правильных ответов по крайней мере до 0.75
* Проверить *accuracy* на тестовой выборке
* Выполнить бонусное задание

------

## 1. Общая информация о данных

Импортируем необходимые модули и библиотеки.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Прочитаем файл `users_behavior.csv` и сохраним его в переменной `data`.

In [4]:
data = pd.read_csv('/datasets/users_behavior.csv')

Выведем на экран первые 5 строк датасета и изучим информацию о данных.

In [7]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В датасете имеется 5 признаков: *calls*, *minutes*, *messages*, *mb_used*, и целевой категориальный признак *is_ultra*. Таким образом в ходе проекта будет решаться задача бинарной классификации.

------

## 2. Подготовка обучающей, валидационной и тестовой выборок

Разобьем исходный датасет на обучающую, валидационную и тестовую выборки в соотношении 3:1:1 соответственно. 

Для этого используем функцию `train_test_split`. Так как функция возвращает только два набора данных, то сначала получим тестовую выборку, которая будет составлять 20% от исходного датасета. Следующим этапом получим тестовую и валиадационную выборки. Параметр `random state` примем равным 12.

In [9]:
data_step, data_test = train_test_split(data, test_size=0.2, random_state=12)

In [10]:
data_train, data_valid = train_test_split(data_step, test_size=0.25, random_state=12)

Проверим что получилось.

In [11]:
data_test.shape

(643, 5)

In [12]:
data_train.shape

(1928, 5)

In [13]:
data_valid.shape

(643, 5)

1928 : 643 : 643 = 3 : 1 : 1. Выборки подготовлены.

------

## 3. Исследование моделей

Для решения задачи классификации исследуем следующие модели: **решающее дерево**, **случайный лес** и **логистическую регрессию**.

Сохраним признаки обучающей и валидационной выборок в переменных `features_train` и `features_valid`. Также сохраним целевые признаки обучающей и валидационной выборок в переменных `target_train` и `target_valid`. 

In [14]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

### 3.1 Решающее дерево

Исследуем модель **решающее дерево** на глубине дерева от 1 до 6. Для исследования всех моделей примем параметр *random_state* равным 12.

In [15]:
for i in range(1, 7):
    model_decTree = DecisionTreeClassifier(max_depth=i, random_state=12)
    model_decTree.fit(features_train, target_train)
    predictions_decTree = model_decTree.predict(features_valid)
    
    print("max_depth =", i, ": ", end='')
    print(accuracy_score(target_valid, predictions_decTree))

max_depth = 1 : 0.7573872472783826
max_depth = 2 : 0.7698289269051322
max_depth = 3 : 0.7807153965785381
max_depth = 4 : 0.7916018662519441
max_depth = 5 : 0.7962674961119751
max_depth = 6 : 0.7916018662519441


Лучший результат доли правильных ответов - **79,6%** получился при глубине дерева, равной 5.

### 3.2 Случайный лес

Исследуем модель **случайный лес** при значениях гиперпараметра *n_estimators* 10, 20, 30, 40, 50 и 60. 

In [16]:
for estim in range(10, 61, 10):
    model_randomForest = RandomForestClassifier(n_estimators=estim, max_depth=10, random_state=12)
    model_randomForest.fit(features_train, target_train)
    predictions_randomForest = model_randomForest.predict(features_valid)
    
    print("n_estimators =", estim, ":", accuracy_score(target_valid, predictions_randomForest))

n_estimators = 10 : 0.7993779160186625
n_estimators = 20 : 0.8087091757387247
n_estimators = 30 : 0.8055987558320373
n_estimators = 40 : 0.8055987558320373
n_estimators = 50 : 0.8040435458786936
n_estimators = 60 : 0.80248833592535


Лучший результат доли правильных ответов - **80,6%** получился при *n_estimators = 40*.

### 3.3 Логистическая регрессия

Исследуем модель **логистическая регрессия**, меняя гиперпараметры `solver` и `penalty`.

In [17]:
model_regression = LogisticRegression(random_state=12, solver='newton-cg', penalty='l2')
model_regression.fit(features_train, target_train)
predictions_regression = model_regression.predict(features_valid)
accuracy_score(target_valid, predictions_regression)

/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:426: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWa

0.749611197511664

In [18]:
model_regression = LogisticRegression(random_state=12, solver='saga', penalty='l1')
model_regression.fit(features_train, target_train)
predictions_regression = model_regression.predict(features_valid)
accuracy_score(target_valid, predictions_regression)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.7107309486780715

In [19]:
model_regression = LogisticRegression(random_state=12, solver='saga', penalty='elasticnet',l1_ratio=1)
model_regression.fit(features_train, target_train)
predictions_regression = model_regression.predict(features_valid)
accuracy_score(target_valid, predictions_regression)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.7107309486780715

Лучший результат доли правильных ответов - **75%**.

Самое высокое качество получилось у модели - случайный лес. Хотя эта модель работает медленнее, чем логистическая регрессия или решающее дерево, в данном случае этот критерий не так важен.

Для проверки на тестовой выборке выберем модель **случайный лес**.

------

## 4. Проверка модели на тестовой выборке

Сохраним признаки тестовой выборки в переменной `features_test`. Сохраним целевой признак тестовой выборки в переменной `target_test`. Для обучения выбранной модели возьмем датасет `data_step`, который состоит из тренировочного и валидационного наборов данных.

In [20]:
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']
features_step = data_step.drop(['is_ultra'], axis=1)
target_step = data_step['is_ultra']

Для исследования модели **случайный лес** на тестовой выборке выберем следующие гиперпараметры: *n_estimators=50, max_depth=10, random_state=12*.

In [21]:
model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=12)
model.fit(features_step, target_step)
predictions = model.predict(features_test)
accuracy_score(target_test, predictions)

0.7620528771384136

Доля правильных ответов - **76,2%**. Цель проекта достигнута.

------

## 5. Проверка модели на адекватность

Для проверки модели на адекватность, необходимо сравнить ее со случайной. Посмотрим какую долю правильных ответов может дать случайная модель. Для этого нужно знать распределения 0 и 1 в тестовой выборке.

In [22]:
data_test['is_ultra'].value_counts()

0    413
1    230
Name: is_ultra, dtype: int64

Лучшим результатом предсказания случайной модели будет, если она предскажет только нули.

In [23]:
accuracy_good = 413 / 643

In [24]:
accuracy_good

0.6423017107309487

Наша модель предсказывает точнее, чем случайная: 76,2% против 64,2%. Это свидетельствует о ее адекватности.

## Выводы

В ходе проекта нужно было построить качественную модель для задачи классификации. По признакам количество звонков, длительность разговора, количество сообщений и объем трафика модель должна подбирать подходящий тариф для пользователя.

В процессе работы исходный датасет поделен на три выборки: обучающую, валидационную и тестовую. Для решения задачи использовались модели решающее дерево, случайный лес и логистическая регрессия. Исследование проводилось при различных значениях гиперпараметров. Лучший результат получился у модели случайный лес.

После проверки этой модели на тестовой выборке доля правильных ответов составила 76,2%. Сравнение модели случайный лес со случайной моделью говорит о высоком качестве получившейся модели.

------